# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 28 2023 8:30AM,258290,10,9212086,"Snap Medical Industries, LLC",Released
1,Feb 28 2023 8:30AM,258290,10,9212087,"Snap Medical Industries, LLC",Released
2,Feb 28 2023 8:30AM,258290,10,9212088,"Snap Medical Industries, LLC",Released
3,Feb 28 2023 8:30AM,258290,10,9212089,"Snap Medical Industries, LLC",Released
4,Feb 28 2023 8:30AM,258290,10,9212090,"Snap Medical Industries, LLC",Released
5,Feb 28 2023 8:30AM,258290,10,9212091,"Snap Medical Industries, LLC",Released
6,Feb 28 2023 8:30AM,258290,10,9212093,"Snap Medical Industries, LLC",Released
7,Feb 28 2023 8:30AM,258290,10,9212094,"Snap Medical Industries, LLC",Released
8,Feb 28 2023 8:30AM,258290,10,9213311,"Snap Medical Industries, LLC",Released
9,Feb 28 2023 8:30AM,258290,10,9213312,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,258283,Released,1
47,258284,Released,23
48,258287,Released,1
49,258288,Released,1
50,258290,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
258283,NaN,NaN,NaN,1.0
258284,NaN,NaN,NaN,23.0
258287,NaN,NaN,NaN,1.0
258288,NaN,NaN,NaN,1.0
258290,NaN,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
258149,1.0,0.0,15.0,2.0
258153,0.0,0.0,31.0,0.0
258162,0.0,0.0,0.0,58.0
258166,0.0,13.0,4.0,0.0
258188,0.0,0.0,24.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
258149,1.0,0,15,2
258153,0.0,0,31,0
258162,0.0,0,0,58
258166,0.0,13,4,0
258188,0.0,0,24,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,258149,1.0,0,15,2
1,258153,0.0,0,31,0
2,258162,0.0,0,0,58
3,258166,0.0,13,4,0
4,258188,0.0,0,24,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,258149,1.0,,15,2
1,258153,0.0,,31,
2,258162,0.0,,,58
3,258166,0.0,13,4,
4,258188,0.0,,24,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC"
10,Feb 28 2023 8:30AM,258288,10,"Digital Brands, LLC"
11,Feb 27 2023 3:00PM,258287,12,"SD Head USA, LLC"
12,Feb 27 2023 2:47PM,258284,20,"Exact-Rx, Inc."
35,Feb 27 2023 2:45PM,258283,10,"Snap Medical Industries, LLC"
36,Feb 27 2023 2:35PM,258281,19,"AdvaGen Pharma, Ltd"
57,Feb 27 2023 2:31PM,258280,10,ISDIN Corporation
73,Feb 27 2023 2:23PM,258277,10,ISDIN Corporation
116,Feb 27 2023 2:23PM,258278,10,Bio-PRF
124,Feb 27 2023 2:22PM,258279,18,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC",0.0,,,10
1,Feb 28 2023 8:30AM,258288,10,"Digital Brands, LLC",0.0,,,1
2,Feb 27 2023 3:00PM,258287,12,"SD Head USA, LLC",0.0,,,1
3,Feb 27 2023 2:47PM,258284,20,"Exact-Rx, Inc.",0.0,,,23
4,Feb 27 2023 2:45PM,258283,10,"Snap Medical Industries, LLC",0.0,,,1
5,Feb 27 2023 2:35PM,258281,19,"AdvaGen Pharma, Ltd",0.0,,,21
6,Feb 27 2023 2:31PM,258280,10,ISDIN Corporation,0.0,,,16
7,Feb 27 2023 2:23PM,258277,10,ISDIN Corporation,0.0,,,43
8,Feb 27 2023 2:23PM,258278,10,Bio-PRF,0.0,,,8
9,Feb 27 2023 2:22PM,258279,18,"Emersa Waterbox, LLC",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC",10,,
1,Feb 28 2023 8:30AM,258288,10,"Digital Brands, LLC",1,,
2,Feb 27 2023 3:00PM,258287,12,"SD Head USA, LLC",1,,
3,Feb 27 2023 2:47PM,258284,20,"Exact-Rx, Inc.",23,,
4,Feb 27 2023 2:45PM,258283,10,"Snap Medical Industries, LLC",1,,
5,Feb 27 2023 2:35PM,258281,19,"AdvaGen Pharma, Ltd",21,,
6,Feb 27 2023 2:31PM,258280,10,ISDIN Corporation,16,,
7,Feb 27 2023 2:23PM,258277,10,ISDIN Corporation,43,,
8,Feb 27 2023 2:23PM,258278,10,Bio-PRF,8,,
9,Feb 27 2023 2:22PM,258279,18,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC",10,,
1,Feb 28 2023 8:30AM,258288,10,"Digital Brands, LLC",1,,
2,Feb 27 2023 3:00PM,258287,12,"SD Head USA, LLC",1,,
3,Feb 27 2023 2:47PM,258284,20,"Exact-Rx, Inc.",23,,
4,Feb 27 2023 2:45PM,258283,10,"Snap Medical Industries, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC",10.0,NaN,NaN
1,Feb 28 2023 8:30AM,258288,10,"Digital Brands, LLC",1.0,NaN,NaN
2,Feb 27 2023 3:00PM,258287,12,"SD Head USA, LLC",1.0,NaN,NaN
3,Feb 27 2023 2:47PM,258284,20,"Exact-Rx, Inc.",23.0,NaN,NaN
4,Feb 27 2023 2:45PM,258283,10,"Snap Medical Industries, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC",10.0,0.0,0.0
1,Feb 28 2023 8:30AM,258288,10,"Digital Brands, LLC",1.0,0.0,0.0
2,Feb 27 2023 3:00PM,258287,12,"SD Head USA, LLC",1.0,0.0,0.0
3,Feb 27 2023 2:47PM,258284,20,"Exact-Rx, Inc.",23.0,0.0,0.0
4,Feb 27 2023 2:45PM,258283,10,"Snap Medical Industries, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5939318,332.0,119.0,0.0
12,516532,1.0,1.0,0.0
15,774611,37.0,24.0,83.0
16,516473,1.0,1.0,0.0
18,258279,1.0,0.0,0.0
19,2065842,27.0,20.0,13.0
20,258284,23.0,0.0,0.0
21,258234,1.0,0.0,0.0
22,516510,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5939318,332.0,119.0,0.0
1,12,516532,1.0,1.0,0.0
2,15,774611,37.0,24.0,83.0
3,16,516473,1.0,1.0,0.0
4,18,258279,1.0,0.0,0.0
5,19,2065842,27.0,20.0,13.0
6,20,258284,23.0,0.0,0.0
7,21,258234,1.0,0.0,0.0
8,22,516510,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,332.0,119.0,0.0
1,12,1.0,1.0,0.0
2,15,37.0,24.0,83.0
3,16,1.0,1.0,0.0
4,18,1.0,0.0,0.0
5,19,27.0,20.0,13.0
6,20,23.0,0.0,0.0
7,21,1.0,0.0,0.0
8,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,332.0
1,12,Released,1.0
2,15,Released,37.0
3,16,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,83.0,0.0,0.0,13.0,0.0,0.0,0.0
Executing,119.0,1.0,24.0,1.0,0.0,20.0,0.0,0.0,0.0
Released,332.0,1.0,37.0,1.0,1.0,27.0,23.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,0.0,0.0,83.0,0.0,0.0,13.0,0.0,0.0,0.0
1,Executing,119.0,1.0,24.0,1.0,0.0,20.0,0.0,0.0,0.0
2,Released,332.0,1.0,37.0,1.0,1.0,27.0,23.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,0.0,0.0,83.0,0.0,0.0,13.0,0.0,0.0,0.0
1,Executing,119.0,1.0,24.0,1.0,0.0,20.0,0.0,0.0,0.0
2,Released,332.0,1.0,37.0,1.0,1.0,27.0,23.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()